In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import liana as li
import matplotlib.pyplot as plt
import scipy.sparse as sp



from matplotlib import colors
%matplotlib inline

from typing import Optional, Literal

In [ ]:
adata = sc.read("./citeseq_mdata_allsamples_filtered_fine_clustering.h5ad")

In [ ]:
adata

In [ ]:
print(adata.raw)

In [ ]:
li.mt.show_methods()


In [ ]:
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

In [ ]:
adata.obs['celltype_midres'] = adata.obs['celltype_hires']

In [ ]:
adata.obs['celltype_midres'] = adata.obs['celltype_midres'].replace({
    'Tcell_CD4+blood': 'T_CD4+',
    'Tcell_CD4+cyto': 'T_CD4+',
    'Tcell_CD4+exh': 'T_CD4+',
    'Tcell_CD4+prol': 'T_CD4+',
    'Tcell_CD4+tr': 'T_CD4+',
    'Tcell_CD8+blood': 'T_CD8+',
    'Tcell_CD8+eff': 'T_CD8+',
    'Tcell_CD8+prol': 'T_CD8+',
    'Tcell_CD8+tr': 'T_CD8+',
    'MAITcell':'T_CD8+',
    'Tcell_reg': 'Treg',
    'DC1_prol': 'DC1',
    'DC2_prol': 'DC2',
    'CD39-CD103+ NK':'CD39- NK'
})

In [ ]:
adata = adata[~adata.obs['celltype_midres'].isin(['CCT','vCTB','STB'])]

In [ ]:
adata.obs['celltype_midres'].cat.categories

In [ ]:
sc.pl.umap(adata, color='celltype_midres')

In [ ]:
resource = li.rs.select_resource('consensus')
resource.head()

In [ ]:
resource

In [ ]:
resource[resource['ligand'] == "IFNG"]

In [ ]:
new_pairs = pd.DataFrame({
    'ligand': ['PTGES'] * 4,
    'receptor': ['PTGER1', 'PTGER2', 'PTGER3', 'PTGER4']
})

# 2. Append them to your existing df
resource = pd.concat([resource, new_pairs], ignore_index=True)

In [ ]:
new_pairs = pd.DataFrame({
    'ligand': ['HSD11B1'],
    'receptor': ['NR3C1']
})

# 2. Append them to your existing df
resource = pd.concat([resource, new_pairs], ignore_index=True)

In [ ]:
new_pairs = pd.DataFrame({
    'ligand': ['GDF15'],
    'receptor': ['ITGAL']
})

# 2. Append them to your existing df
resource = pd.concat([resource, new_pairs], ignore_index=True)

In [ ]:
resource

In [ ]:
raw = adata.raw


gene = 'ARG1'
if gene not in raw.var_names:
    raise KeyError(f"{gene!r} not found in raw.var_names")
gi = raw.var_names.get_loc(gene)


fn1_vec = raw.X[:, gi]


if sp.issparse(fn1_vec):
    fn1_arr = fn1_vec.toarray().ravel()
else:
    fn1_arr = fn1_vec.ravel()


fn1_series = pd.Series(fn1_arr, index=adata.obs_names, name='FN1_raw')


df = adata.obs[['celltype_midres']].join(fn1_series)


mean_by_type = df.groupby('celltype_midres')['FN1_raw'].mean()

print(mean_by_type)

In [ ]:
print(adata.raw.X)

In [ ]:
cellphonedb(adata,
            groupby='celltype_midres',
            resource = resource,
            expr_prop=0.20,
            verbose=True, key_added='cpdb_res')

In [ ]:
adata.uns['cpdb_res'].head()

In [ ]:
adata.uns['cpdb_res']

In [ ]:
df = adata.uns['cpdb_res']


raw = adata.raw
celltypes = adata.obs['celltype_midres']  

genes = pd.unique(df[['ligand','receptor']].values.ravel())

scaled_means = {}
for g in genes:
    if g not in raw.var_names:
        raise KeyError(f"{g!r} not found in adata.raw.var_names")
    gi = raw.var_names.get_loc(g)
    vec = raw.X[:, gi]
    arr = vec.toarray().ravel() if sp.issparse(vec) else vec.ravel()
    # mean per cell type
    means = pd.Series(arr, index=celltypes).groupby(level=0).mean()
    # min–max scale
    scaled = (means - means.min()) / (means.max() - means.min())
    scaled_means[g] = scaled


adata.uns['cpdb_res']['ligand_scaled_means']   = df.apply(lambda row: scaled_means[row['ligand']].loc[row['source']],   axis=1)
adata.uns['cpdb_res']['receptor_scaled_means'] = df.apply(lambda row: scaled_means[row['receptor']].loc[row['target']], axis=1)

In [ ]:
df = adata.uns['cpdb_res']

df[df['ligand'].isin(['PTGES','HSD11B1','IFNG','GDF15'])]

In [ ]:
adata.uns['cpdb_res'].head()

In [ ]:
import plotnine as p9
from typing import Union, List, Tuple
import anndata as ad
import pandas as pd

from liana.plotting._common import _prep_liana_res, _get_top_n, _check_var, _filter_by
from liana._docs import d
from liana._constants import Keys as K, DefaultValues as V

@d.dedent
def tileplot(adata: ad.AnnData = None,
             liana_res: pd.DataFrame = None,
             fill: str = None,
             label: str = None,
             label_fun: callable = None,
             source_labels: Union[str, List[str]] = None,
             target_labels: Union[str, List[str]] = None,
             ligand_complex: Union[str, List[str]] = None,
             receptor_complex: Union[str, List[str]] = None,
             uns_key: str = K.uns_key,
             top_n: int = None,
             orderby: str = None,
             orderby_ascending: bool = False,
             orderby_absolute: bool = True,
             filter_fun: callable = None,
             source_title=None,
             target_title=None,
             cmap: str = V.cmap,
             figure_size: Tuple[float, float] = (5, 5),
             label_size: int = 12,  # (Unused in this version)
             return_fig: bool = V.return_fig
             ):
    """
    Tileplot interactions by source and target cells, now displaying the props values as circles.
    The bubble fill color reflects the means values (using the `fill` column) while the bubble size
    reflects the props values (using the `label` column).

    Parameters
    ----------
    %(adata)s
    %(liana_res)s
    fill
        Column in `liana_res` to define the bubble fill (e.g., the means values).
    label
        Column in `liana_res` to define the bubble size (e.g., the props values).
    label_fun
        Callable to apply to the `label` column.
    %(source_labels)s
    %(target_labels)s
    %(ligand_complex)s
    %(receptor_complex)s
    %(uns_key)s
    %(top_n)s
    %(orderby)s
    %(orderby_ascending)s
    %(orderby_absolute)s
    %(filter_fun)s
    source_title
        Title for the source facet. Default is 'Source'.
    target_title
        Title for the target facet. Default is 'Target'.
    %(cmap)s
    label_size
        (Not used in this version; bubble size is determined by the `label` values.)
    %(figure_size)s
    %(return_fig)s

    Returns
    -------
    A `plotnine.ggplot` instance displaying bubbles where the fill color indicates the means value and
    the bubble size reflects the props value.
    """
    liana_res = _prep_liana_res(adata=adata,
                                liana_res=liana_res,
                                source_labels=source_labels,
                                target_labels=target_labels,
                                ligand_complex=ligand_complex,
                                receptor_complex=receptor_complex,
                                uns_key=uns_key)

    liana_res = _filter_by(liana_res, filter_fun)
    liana_res = _get_top_n(liana_res, top_n, orderby, orderby_ascending, orderby_absolute)

    relevant_cols = [col for col in liana_res.columns if col.endswith(fill) | col.endswith(label)]

    ligand_stats = _entity_stats(liana_res,
                                 entity='ligand',
                                 entity_type='source',
                                 relevant_cols=relevant_cols,
                                 type_title=source_title)

    _check_var(ligand_stats, var=fill, var_name='fill')
    _check_var(ligand_stats, var=label, var_name='label')

    receptor_stats = _entity_stats(liana_res,
                                   entity='receptor',
                                   entity_type='target',
                                   relevant_cols=relevant_cols,
                                   type_title=target_title)

    liana_res = pd.concat([ligand_stats, receptor_stats])

    if label_fun is not None:
        liana_res[label] = liana_res[label].apply(label_fun)


    p = (
        p9.ggplot(liana_res, p9.aes(x='cell_type', y='interaction')) +
        p9.geom_point(p9.aes(size=label, fill=fill), shape='o', color='white') +
        p9.facet_grid('~ type', scales='free') +
        p9.theme_bw(base_size=14) +
        p9.theme(
            axis_text_x=p9.element_text(angle=90),
            figure_size=figure_size,
            strip_background=p9.element_rect(colour="black", fill="#fdfff4"),
        ) +
        p9.scale_fill_cmap(
            cmap,
            limits=(0, 1),
            breaks=[0,0.25,0.5,0.75, 1]
        ) +
        p9.scale_size_continuous(range=(0, 10), limits=(0, 1)) +
        p9.labs(x='Cell type', y='Interaction', fill=str.capitalize(fill), size=str.capitalize(label))
    )

    if return_fig:
        return p

    p.draw()

def _entity_stats(liana_res, entity, entity_type, relevant_cols, type_title=None):
    entity_stats = liana_res[['interaction', f"{entity}_complex", entity_type, *relevant_cols]].copy()
    if type_title is None:
        type_title = entity_type.capitalize()
    entity_stats = entity_stats.rename(columns={entity_type: 'cell_type'}).assign(type=type_title)
    entity_stats.columns = entity_stats.columns.str.replace(entity + '_', '')
    return entity_stats


## Figure 5F

In [ ]:
my_plot = tileplot(
    adata=adata,
    # fill color represents the means values
    fill='means',
    # bubble size represents the props values
    label='props',
    #label_fun=lambda x: f'{x:.2f}',
    top_n=50,
    orderby='cellphone_pvals',
    cmap='Blues', 
    orderby_ascending=True,
    ligand_complex=['IL10', 'MIF', 'BAG6', 'PVR', 'CD274', 'PTGES', 'HLA-G', 'TNFSF10','GDF15'],
    source_labels=['CD39- NK', 'decBAM1', 'decBAM2', 'EVT'],
    target_labels=['T_CD4+', 'T_CD8+', 'CD39- NK', 'DC1', 'decPAM2'],
    uns_key='cpdb_res', 
    source_title='Ligand',
    target_title='Receptor',
    figure_size=(8, 7.5)
)

my_plot.save("20250624_liana_hubdissolution.svg", format="svg", width=8, height=8.25, units="in")
my_plot

In [ ]:
my_plot = tileplot(
    adata=adata,
    # fill color represents the means values
    fill='means',
    # bubble size represents the props values
    label='props',
    #label_fun=lambda x: f'{x:.2f}',
    top_n=50,
    orderby='cellphone_pvals',
    cmap='Blues', 
    orderby_ascending=True,
    ligand_complex=['IL10', 'MIF', 'BAG6', 'PVR', 'CD274', 'PTGES', 'HLA-G', 'TNFSF10','GDF15'],
    source_labels=['CD39- NK', 'decBAM1', 'decBAM2', 'EVT'],
    uns_key='cpdb_res',  # NOTE: default is 'liana_res'
    source_title='Ligand',
    target_title='Receptor',
    figure_size=(16, 7.5)
)

my_plot.save("20250624_liana_hubdissolution_all.svg", format="svg", width=8, height=8.25, units="in")
my_plot

In [ ]:
my_plot = tileplot(
    adata=adata,
    # fill color represents the means values
    fill='means',
    # bubble size represents the props values
    label='props',
    #label_fun=lambda x: f'{x:.2f}',
    top_n=50,
    orderby='cellphone_pvals',
    cmap='Blues', 
    orderby_ascending=True,
    ligand_complex=['IL10', 'MIF', 'BAG6', 'PVR', 'CD274', 'PTGES', 'HLA-G', 'TNFSF10','GDF15'],
    source_labels=['CD39- NK', 'decBAM1', 'decBAM2', 'EVT'],
    target_labels=['T_CD4+', 'T_CD8+', 'CD39- NK', 'DC1', 'decPAM2'],
    uns_key='cpdb_res',  # NOTE: default is 'liana_res'
    source_title='Ligand',
    target_title='Receptor',
    figure_size=(8, 7.5)
)

my_plot.save("20250624_liana_hubdissolution.svg", format="svg", width=8, height=8.25, units="in")
my_plot

In [ ]:
df = adata.uns['cpdb_res']

df[df['ligand'].isin(['IFNG'])]

## Figure 4D

In [ ]:
my_plot = tileplot(adata = adata,
                         # NOTE: fill & label need to exist for both
                         # ligand_ and receptor_ columns
                         fill='means',
                         label='props',
                       cmap='Blues',
                         #label_fun=lambda x: f'{x:.2f}',
                         top_n=50,
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                        ligand_complex = ['CXCL9','CXCL10','CXCL11','XCL1','CXCL16','TNFSF13B','IFNG'],
                         source_labels=['decPAM2','CD39- NK','DC1'],
                        target_labels=['T_CD4+','T_CD8+','CD39- NK','DC1','decPAM2'],
                        uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(8, 6)
                         )
my_plot.save("20250624_liana_hubcreation.svg", format="svg", width=8, height=6, units="in")
my_plot

In [ ]:
my_plot = tileplot(adata = adata,
                         # NOTE: fill & label need to exist for both
                         # ligand_ and receptor_ columns
                         fill='means',
                         label='props',
                       cmap='Blues',
                         #label_fun=lambda x: f'{x:.2f}',
                         top_n=50,
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                        ligand_complex = ['CXCL9','CXCL10','CXCL11','XCL1','CXCL16','IFNG'],
                         source_labels=['decPAM2','CD39- NK','DC1'],
                        target_labels=['T_CD4+','T_CD8+','CD39- NK','DC1','decPAM2','Mono','BCell','CD39+ NK','DC2','decBAM2','decBAM1','decPAM1',
                                      'MAST'],
                        uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(10, 6)
                         )
my_plot.save("20250624_liana_hubcreation_all.svg", format="svg", width=8, height=6, units="in")
my_plot

In [ ]:
my_plot = tileplot(adata = adata,
                         # NOTE: fill & label need to exist for both
                         # ligand_ and receptor_ columns
                         fill='means',
                         label='props',
                       cmap='Blues',
                         #label_fun=lambda x: f'{x:.2f}',
                         top_n=50,
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                        ligand_complex = ['CXCL9','CXCL10','CXCL11','XCL1','CXCL16','IFNG'],
                         source_labels=['decPAM2','CD39- NK','DC1'],
                        target_labels=['T_CD4+','T_CD8+','CD39- NK','DC1','decPAM2','Mono','BCell','CD39+ NK','DC2','decBAM2','decBAM1','decPAM1',
                                      'MAST'],
                        uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(16, 6)
                         )
my_plot.save("20250624_liana_hubcreation_all2.svg", format="svg", width=16, height=6, units="in")
my_plot